#**Introducción a Machine Learning**

#_Desafío 2: Expansiones basales_

##**Francisca Pinto**
##**03 de octubre de 2021**

Comentarios:
1. Se inicia la instalación de módulos para la ejecución de celdas en Google Drive. Se dejarán las celdas relativas a este proceso con "<code>#</code>".
2. Se inicia con los imports y la incorporación del DataFrame.

In [15]:
#!pip install ipython-autotime
#!pip install pygam

time: 6.28 s (started: 2021-10-03 19:37:18 +00:00)


In [16]:
#tiempo de ejecución
%load_ext autotime

#dataframe y arrays
import pandas as pd
import numpy as np

#gráficos

import matplotlib.pyplot as plt
import seaborn as sns

#gama matplotlib para facilitar escritura de código
%matplotlib inline

#regresión lineal

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#estandarización de atributos
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

#sklearn análisis resumido
from sklearn.pipeline import Pipeline

#análisis de nulos dedataframe
import missingno as msno

#expansiones basales y no-linealidades
from pygam import LinearGAM, s

#quitar warnings
#import warnings
#warnings.filterwarnings(action = "ignore")

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 15.2 ms (started: 2021-10-03 19:37:24 +00:00)


In [17]:
plt.style.use("seaborn") #tipo de gráfico, estilo
plt.rcParams["figure.figsize"] = (5,5) #escalar gráficos
plt.rcParams["figure.dpi"] = 150 #DPI gráficos (también modifica tamaño)

time: 3.24 ms (started: 2021-10-03 19:37:24 +00:00)


In [18]:
#AUX: montar archivos y carpetas desde Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.87 ms (started: 2021-10-03 19:37:24 +00:00)


In [19]:
colab_path = "/content/drive/MyDrive/ADL/3_Intro ML/Desafío 2_ Expansiones basales/compresive_strength_concrete.csv"
df = pd.read_csv(colab_path)
df.reset_index(drop = True, inplace = True)
df.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


time: 42.5 ms (started: 2021-10-03 19:37:24 +00:00)


In [20]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1030 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1030 non-null   float64
 7   Age (day)                                              1030 non-null   int64  
 8   Concrete compressive strength(MPa, megapascals)  

Comentarios:
1. No se observan datos nulos: el tipo de dato en cada atributo implica que solo hay datos numéricos en todas las columnas.
2. Para observar mejor la distribución de los datos se aplicará el método <code>describe</code>.

In [21]:
df.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


time: 119 ms (started: 2021-10-03 19:37:24 +00:00)


Comentarios:

1. Existen variables con rangos en decenas (como <code>Superplasticizer</code> o <code>Concrete compressive strength</code>), mientras que otros llegan a miles (<code>Coarse Aggregate</code>), habiendo disparidad entre ellos.
2. Se separarán los conjuntos para el entrenamiento y validación de la máquina.
3. Se estandarizarán las variables para evitar problemas de interpretación del modelo con <code>StandardScaler</code>, y se realizará el primer modelo.

In [22]:
X = df.iloc[:, : -1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 374)

time: 6.64 ms (started: 2021-10-03 19:37:24 +00:00)


In [23]:
std = StandardScaler().fit(X_train)
X_train_scaled = std.transform(X_train)
X_test_scaled = std.transform(X_test)

time: 7.58 ms (started: 2021-10-03 19:37:24 +00:00)


In [24]:
gam1 = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7), fit_intercept = True).fit(X_train_scaled, y_train)
gam1.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     81.9243
Link Function:                     IdentityLink Log Likelihood:                                 -3506.8302
Number of Samples:                          824 AIC:                                             7179.5089
                                                AICc:                                             7198.316
                                                GCV:                                               33.6396
                                                Scale:                                             27.6785
                                                Pseudo R-Squared:                                   0.9122
Feature Function                  Lam

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  


Comentarios:
1. Todos los índices <code>lambda</code> son iguales.
2. Los <code>p-value</code> están muy por debajo del umbral <code>0.05</code> para un nivel de certeza de un 95%, por lo que los interceptos calculados por el modelo pueden considerarse válidos.
3. Se genera el vector objetivo predicho y se calcularán las métricas R2, MSE (Mean Squared Error) y MAE (Median Absolute Error).

In [25]:
yhat_gam1 = gam1.predict(X_test_scaled)

time: 25.8 ms (started: 2021-10-03 19:37:24 +00:00)


In [26]:
def report_visualization_gam(model, Xtest, ytest, yhat):
  print(f"Reporte de métricas de Modelo:")
  print(f"-----------------------------------------")
  print(f"R-squared: {r2_score(ytest, yhat)}")
  print(f"Mean Squared Error (MSE): {mean_squared_error(ytest, yhat)}")
  print(f"Median Absolute Error (MAE): {median_absolute_error(ytest, yhat)}\n")

time: 3.56 ms (started: 2021-10-03 19:37:24 +00:00)


In [27]:
report_visualization_gam(gam1, X_test_scaled, y_test, yhat_gam1)

Reporte de métricas de Modelo:
-----------------------------------------
R-squared: 0.8993395719166848
Mean Squared Error (MSE): 26.047735548279505
Median Absolute Error (MAE): 3.450959068060726

time: 13.4 ms (started: 2021-10-03 19:37:24 +00:00)


Comentarios:
1. MSE es muy alto en relación al rango de la resistencia a compresión, el modelo así es no es muy útil con estas condiciones: los rangos de resistencia en el tercer intercuartil llegan a 46MPa (MegaPascales), por lo que una variación de 26MPa es demasiado grande y no permite evaluar adecuadamente los resultados de la viga testeada.

In [28]:
lams = np.logspace(-3, 3, 3)
lams = [lams] * len(X_train.columns)
gam2 = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7), fit_intercept = True).fit(X_train_scaled, y_train)
gam2.gridsearch(X_train_scaled, y_train, lam = lams)

yhat_gam2 = gam2.predict(X_test_scaled)
gam2.summary()

100% (6561 of 6561) |####################| Elapsed Time: 0:20:48 Time:  0:20:48


LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     80.0728
Link Function:                     IdentityLink Log Likelihood:                                 -3479.2231
Number of Samples:                          824 AIC:                                             7120.5919
                                                AICc:                                            7138.5286
                                                GCV:                                               32.3557
                                                Scale:                                             26.7503
                                                Pseudo R-Squared:                                   0.9149
Feature Function                  Lam

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  import sys


In [29]:
report_visualization_gam(gam2, X_test_scaled, y_test, yhat_gam2)

Reporte de métricas de Modelo:
-----------------------------------------
R-squared: 0.902960303508229
Mean Squared Error (MSE): 25.110804712760057
Median Absolute Error (MAE): 3.395634915418259

time: 9.27 ms (started: 2021-10-03 19:58:13 +00:00)


Comentarios:

1. La edad es el parámetro más castigado, el Lambda es el menor de todos, una milésima parte de los valores, en comparación con el resto de atributos. La resistencia a compresión del hormigón, se calcula al día 28 por consideraciones constructivas, y según la literatura [1], a esta fecha se ha alcanzado un 65% (aunque esta cifra puede variar ligeramente) de la resistencia total que alcanzará la pieza. Dado ello, la edad de hormigón, luego de esta fecha, tiene una baja influencia en el comportamiento a compresión.
2. Los valores lambda son todos iguales, salvo en el caso de la edad.
3. Los <code>p-value</code> están por debajo de 0.05, asegurando la confiabilidad de los resultados de cada parámetro.
4. El valor de MAE bajó considerablemente y ahora se acerca a los 3.4MPa, lo que permite dar fiabilidad a la evaluación de las vigas testeadas, comparando los resultados con los requerimientos de cada caso.

Fuentes:
1. <code>https://riunet.upv.es/bitstream/handle/10251/12793/Evoluci%C3%B3n%20resistencia%20hormig%C3%B3n.pdf</code>